In [ ]:
pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=034c2d042332c7c3e2c58ea5b4310d112062fe494cc631b14c81959f5c5855c3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


# New Section

In [ ]:
import os
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [ ]:
df=pd.read_csv('/content/df_concat.csv')

In [ ]:
df_copy = df.copy(deep=True)

In [ ]:
df.columns

Index(['Unnamed: 0', 'type', 'title', 'director', 'cast', 'country',
       'date_added', 'release_year', 'rating', 'duration', 'listed_in',
       'description', 'month_added', 'month_name_added', 'year_added',
       'type_of_movie'],
      dtype='object')

In [ ]:
df.head()

,Unnamed: 0,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added,type_of_movie
0,0,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",United States,2021-11-26,2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,11,November,2021,Animation
1,1,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",United States,2021-11-26,1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,11,November,2021,Comedy
2,2,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,11,November,2021,Animation
3,3,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",United States,2021-11-26,2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",11,November,2021,Musical
4,4,TV Show,The Beatles: Get Back,No Data,"John Lennon, Paul McCartney, George Harrison, ...",United States,2021-11-25,2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,11,November,2021,Docuseries


In [ ]:
df.replace('No Data', 'no_data', inplace=True)

In [ ]:
df.drop(columns=['Unnamed: 0','date_added','duration', 'month_added', 'month_name_added', 'year_added', 'type_of_movie'],inplace=True)

In [ ]:
df.head()

,type,title,director,cast,country,release_year,rating,listed_in,description
0,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",United States,2016,TV-G,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",United States,1988,PG,Comedy,Santa Claus passes his magic bag to a new St. ...
2,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2011,TV-G,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",United States,2021,TV-PG,Musical,"This is real life, not just fantasy!"
4,TV Show,The Beatles: Get Back,no_data,"John Lennon, Paul McCartney, George Harrison, ...",United States,2021,NaN,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [ ]:
def create_textual_representation(row):
    txt= f"""Type: {row['type']}
Director: {row['director']}
Cast: {row['cast']}
Release Year: {row['release_year']}
Country: {row['country']}
Genre: {row['listed_in']}

Description: {row['cleaned_description']}"""

    return txt

In [ ]:
def clean_text(text):
    # Remove punctuation
    punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    text = ''.join([char for char in text if char not in punctuation])

    # Convert to lowercase
    text = text.lower()

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

In [ ]:
def trim_cast(cast_str):
    """Trims the cast string to only the first 3 actors and returns them as a list with spaces replaced by underscores."""
    if isinstance(cast_str, str):
        actors = cast_str.split(',')
        return [actor.strip().replace(' ', '_') for actor in actors[:3]]
    return []

In [ ]:
df['cast'] = df['cast'].apply(lambda x: ', '.join(trim_cast(x)))
df['director'] = df['director'].apply(lambda x: ', '.join(trim_cast(x)))

In [ ]:
df['cleaned_description'] = df['description'].apply(clean_text)

In [ ]:
df['textual_representation']=df.apply(create_textual_representation, axis=1)

In [ ]:
print(df['textual_representation'].values[0])

Type: Movie
Director: Alonso_Ramirez_Ramos, Dave_Wasson
Cast: Chris_Diamantopoulos, Tony_Anselmo, Tress_MacNeille
Release Year: 2016
Country: United States
Genre: Animation, Family

Description: join mickey and the gang as they duck the halls


In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["textual_representation"].apply(lambda x: len(tokenizer.encode(x)))

In [ ]:
total_tokens = df['n_tokens'].sum()
print(f"Total number of tokens: {total_tokens}")
print(f"Total cost of tokens: {total_tokens*0.0001/1000} $")

Total number of tokens: 786295
Total cost of tokens: 0.0786295 $


In [ ]:
df.head()

,type,title,director,cast,country,release_year,rating,listed_in,description,cleaned_description,textual_representation,n_tokens
0,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso_Ramirez_Ramos, Dave_Wasson","Chris_Diamantopoulos, Tony_Anselmo, Tress_MacN...",United States,2016,TV-G,"Animation, Family",Join Mickey and the gang as they duck the halls!,join mickey and the gang as they duck the halls,"Type: Movie\nDirector: Alonso_Ramirez_Ramos, D...",70
1,Movie,Ernest Saves Christmas,John_Cherry,"Jim_Varney, Noelle_Parker, Douglas_Seale",United States,1988,PG,Comedy,Santa Claus passes his magic bag to a new St. ...,santa claus passes his magic bag to a new st nic,Type: Movie\nDirector: John_Cherry\nCast: Jim_...,55
2,Movie,Ice Age: A Mammoth Christmas,Karen_Disher,"Raymond_Albert_Romano, John_Leguizamo, Denis_L...",United States,2011,TV-G,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,sid the sloth is on santas naughty list,Type: Movie\nDirector: Karen_Disher\nCast: Ray...,63
3,Movie,The Queen Family Singalong,Hamish_Hamilton,"Darren_Criss, Adam_Lambert, Derek_Hough",United States,2021,TV-PG,Musical,"This is real life, not just fantasy!",this is real life not just fantasy,Type: Movie\nDirector: Hamish_Hamilton\nCast: ...,53
4,TV Show,The Beatles: Get Back,no_data,"John_Lennon, Paul_McCartney, George_Harrison",United States,2021,NaN,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,a threepart documentary from peter jackson cap...,Type: TV Show\nDirector: no_data\nCast: John_L...,70


In [ ]:
df.shape

(10130, 12)

In [ ]:
client = AzureOpenAI(
    azure_endpoint="https://openainew2211.openai.azure.com/",
    api_key="e1ce1acd133b4d69a78cad5c666223de",
    api_version="2024-02-01"
)

In [ ]:
def generate_embeddings(text, model="text-embedding-ada-002"):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
df['ada_v2'] = df["textual_representation"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [ ]:
df.head()

,type,title,director,cast,country,release_year,rating,listed_in,description,cleaned_description,textual_representation,n_tokens,ada_v2
0,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso_Ramirez_Ramos, Dave_Wasson","Chris_Diamantopoulos, Tony_Anselmo, Tress_MacN...",United States,2016,TV-G,"Animation, Family",Join Mickey and the gang as they duck the halls!,join mickey and the gang as they duck the halls,"Type: Movie\nDirector: Alonso_Ramirez_Ramos, D...",70,"[-0.004417451564222574, -0.02741820551455021, ..."
1,Movie,Ernest Saves Christmas,John_Cherry,"Jim_Varney, Noelle_Parker, Douglas_Seale",United States,1988,PG,Comedy,Santa Claus passes his magic bag to a new St. ...,santa claus passes his magic bag to a new st nic,Type: Movie\nDirector: John_Cherry\nCast: Jim_...,55,"[0.0017536545637995005, -0.0326634906232357, -..."
2,Movie,Ice Age: A Mammoth Christmas,Karen_Disher,"Raymond_Albert_Romano, John_Leguizamo, Denis_L...",United States,2011,TV-G,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,sid the sloth is on santas naughty list,Type: Movie\nDirector: Karen_Disher\nCast: Ray...,63,"[-0.00391661049798131, -0.015575433149933815, ..."
3,Movie,The Queen Family Singalong,Hamish_Hamilton,"Darren_Criss, Adam_Lambert, Derek_Hough",United States,2021,TV-PG,Musical,"This is real life, not just fantasy!",this is real life not just fantasy,Type: Movie\nDirector: Hamish_Hamilton\nCast: ...,53,"[-0.0018677887273952365, -0.02954970672726631,..."
4,TV Show,The Beatles: Get Back,no_data,"John_Lennon, Paul_McCartney, George_Harrison",United States,2021,NaN,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,a threepart documentary from peter jackson cap...,Type: TV Show\nDirector: no_data\nCast: John_L...,70,"[-0.003931119572371244, -0.02340295910835266, ..."


In [ ]:
df.to_csv('df_with_embeddings.csv', index=False)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Extract the embeddings as a NumPy array
embeddings_matrix = np.stack(df['ada_v2'].values)

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(embeddings_matrix)

# You now have a cosine similarity matrix where cosine_sim_matrix[i][j] represents the
# cosine similarity between the embedding of item i and item j.

print(cosine_sim_matrix)

[[1.         0.80712338 0.83416796 ... 0.82279704 0.84654992 0.81210712]
 [0.80712338 1.         0.84250461 ... 0.80016005 0.83202288 0.78629445]
 [0.83416796 0.84250461 1.         ... 0.80564048 0.83435901 0.79409077]
 ...
 [0.82279704 0.80016005 0.80564048 ... 1.         0.83621743 0.82755757]
 [0.84654992 0.83202288 0.83435901 ... 0.83621743 1.         0.81765859]
 [0.81210712 0.78629445 0.79409077 ... 0.82755757 0.81765859 1.        ]]


In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df['title'], columns=df['title'])

In [ ]:
cosine_sim_df.head()

title,Duck the Halls: A Mickey Mouse Christmas Special,Ernest Saves Christmas,Ice Age: A Mammoth Christmas,The Queen Family Singalong,The Beatles: Get Back,Becoming Cousteau,Hawkeye,Port Protection Alaska,Secrets of the Zoo: Tampa,A Muppets Christmas: Letters To Santa,...,Zak Storm,Zed Plus,Zenda,Zindagi Gulzar Hai,Zinzana,Zodiac,Zombie Dumb,Zombieland,Zoom,Zubaan
title,,,,,,,,,,,,,,,,,,,,,
Duck the Halls: A Mickey Mouse Christmas Special,1.000000,0.807123,0.834168,0.810547,0.784283,0.800482,0.813161,0.773711,0.792548,0.850717,...,0.811230,0.797384,0.794955,0.762370,0.788737,0.802589,0.810550,0.822797,0.846550,0.812107
Ernest Saves Christmas,0.807123,1.000000,0.842505,0.780857,0.769345,0.761354,0.790955,0.756960,0.755780,0.836986,...,0.794159,0.774040,0.790786,0.767041,0.771957,0.783414,0.786400,0.800160,0.832023,0.786294
Ice Age: A Mammoth Christmas,0.834168,0.842505,1.000000,0.786400,0.755399,0.772180,0.774309,0.746722,0.786799,0.840568,...,0.789827,0.790898,0.799528,0.762349,0.780361,0.801237,0.786025,0.805640,0.834359,0.794091
The Queen Family Singalong,0.810547,0.780857,0.786400,1.000000,0.807295,0.785453,0.793144,0.769655,0.797083,0.817454,...,0.788258,0.802543,0.798481,0.782910,0.795280,0.789028,0.784736,0.800242,0.812716,0.823941
The Beatles: Get Back,0.784283,0.769345,0.755399,0.807295,1.000000,0.783441,0.804394,0.793175,0.803530,0.805328,...,0.807186,0.765266,0.794759,0.775549,0.765019,0.788894,0.786737,0.775822,0.786633,0.798765


In [ ]:
def get_recommendations(title, cosine_sim_df, top_n=10):
  """
  Recommends similar movies based on cosine similarity.

  Args:
      title: The title of the movie for which to find recommendations.
      cosine_sim_df: A DataFrame containing the cosine similarity matrix.
      top_n: The number of top similar movies to return.

  Returns:
      A list of recommended movie titles.
  """
  try:
    # Get the row corresponding to the given title
    similarity_scores = cosine_sim_df.loc[title]

    # Sort the similarity scores in descending order
    similarity_scores = similarity_scores.sort_values(ascending=False)

    # Get the top N similar movies (excluding the movie itself)
    top_similar_movies = similarity_scores[1:top_n + 1].index.tolist()

    return top_similar_movies
  except KeyError:
    print(f"Movie '{title}' not found in the dataset.")
    return []

In [ ]:
movie_title = "The Matrix"
recommended_movies = get_recommendations(movie_title, cosine_sim_df, top_n=5)

print(f"Top 5 recommendations for '{movie_title}':")
for movie in recommended_movies:
  print(movie)

Top 5 recommendations for 'The Matrix':
The Matrix Reloaded
The Matrix Revolutions
Transcendence
Jupiter Ascending
Swordfish


In [ ]:
import pickle

with open('cosine_sim_df.pickle', 'wb') as f:
    pickle.dump(cosine_sim_df, f)

In [ ]:
# Calculate the memory usage in bytes
memory_usage_bytes = cosine_sim_df.memory_usage(deep=True).sum()

# Convert bytes to megabytes
memory_usage_mb = memory_usage_bytes / (1024 ** 2)

print(f"Memory usage of the DataFrame: {memory_usage_mb:.2f} MB")

In [ ]:
cosine_sim_df1 = cosine_sim_df.astype(np.float32)

# Calculate the new memory usage in bytes
new_memory_usage_bytes = cosine_sim_df1.memory_usage(deep=True).sum()

# Convert bytes to megabytes
new_memory_usage_mb = new_memory_usage_bytes / (1024 ** 2)

print(f"New memory usage of the DataFrame: {new_memory_usage_mb:.2f} MB")

In [ ]:
with open('cosine_sim_df.pickle', 'wb') as f:
    pickle.dump(cosine_sim_df1, f)